In [ ]:
# run for first time setup
! pip install ipywidgets
! jupyter nbextension enable --py widgetsnbextension

In [1]:
import sys
sys.path.append("..\\")
import login
login.connect()
import datajoint as dj

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

Connecting mpanze@130.60.53.47:3306


In [12]:
import inspect
out = widgets.Output()

available_schema = dj.list_schemas()
schema_dropdown = widgets.Dropdown(options = available_schema, value = "common_mice")
dj_schema = dj.Schema("common_mouse")
mod = dj.VirtualModule("temp.py", "common_mice", create_schema = False, create_tables=False)

def get_table_names(virtualmodule):
    names = []
    for i in inspect.getmembers(virtualmodule):
        a = getattr(virtualmodule, i[0])
        if type(a) == dj.user_tables.TableMeta:
            names.append(i[0])
    return names

tables = get_table_names(mod)
table_dropdown = widgets.Dropdown(options = tables)

@out.capture(clear_output=True, wait=True)
def schema_update(*args):
    mod = dj.VirtualModule("temp.py", schema_dropdown.value, create_schema = False, create_tables=False)
    tables = get_table_names(mod)
    table_dropdown.options = tables
schema_dropdown.observe(schema_update, "value")

def get_attribute_names(table_class):
    names = list(table_class().heading.attributes.keys())
    return names

@out.capture(clear_output=True, wait=True)
def show_table():
    mod = dj.VirtualModule("temp.py", schema_dropdown.value, create_schema = False, create_tables=False)
    table_class = getattr(mod, table_dropdown.value)
    print(table_class().size_on_disk)
    table_attributes = get_attribute_names(table_class)
    display(table_class())
#table_dropdown.observe(show_table, "value")

update_button = widgets.Button(description="Update")
def update_display(*args):
    show_table()
update_button.on_click(update_display)

w = {}
w['schema'] = schema_dropdown
w['table'] = table_dropdown
w['button'] = update_button
controls = widgets.VBox(list(w.values()))
display(widgets.VBox([controls, out]))

#interact(show_table, schema = schema_dropdown, table = table_dropdown)